In [1]:
# import numpy as np
# import matplotlib
# %matplotlib widget
# import matplotlib.pyplot as plt
# plt.plot([1.6, 2.7])
# plt.title("interactive test")
# plt.xlabel("index")
# ax = plt.gca()
# ax.plot([3.1, 2.2])
# plt.draw()

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
import statsmodels.api as sm

import os
import time
import random
import itertools

from copy import copy
from cycler import cycler
from datetime import date
from scipy.optimize import curve_fit
from statannot import add_stat_annotation

pd.set_option('mode.chained_assignment',None)


import re
dateFormatExcel = re.compile('\d{2}/\d{2}/\d{4}')
dateFormatOk = re.compile('\d{2}-\d{2}-\d{2}')


import matplotlib
import matplotlib.pyplot as plt

%matplotlib widget 
# %matplotlib inline
matplotlib.rcParams.update({'figure.autolayout': True})


SMALLER_SIZE = 10
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# prop_cycle = plt.rcParams['axes.prop_cycle']
# colors = prop_cycle.by_key()['color']
my_default_color_list = ['#ff7f0e', '#1f77b4', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
my_default_color_cycle = cycler(color=my_default_color_list)
plt.rcParams['axes.prop_cycle'] = my_default_color_cycle

In [6]:
mainDir = "C://Users//JosephVermeil//Desktop//ActinCortexAnalysis"
# mainDir = "C://Users//josep//Desktop//ActinCortexAnalysis"
experimentalDataDir = os.path.join(mainDir, "ExperimentalData")
dataDir = os.path.join(mainDir, "DataAnalysis")
figDir = os.path.join(dataDir, "Figures")
todayFigDir = os.path.join(figDir, "Historique//" + str(date.today()))
timeSeriesDataDir = os.path.join(dataDir, "TimeSeriesData")

In [17]:
def modifyExperimentalConditions(save = False):
    # Getting the table
    experimentalDataFile = 'ExperimentalConditions.csv'
    experimentalDataFilePath = os.path.join(experimentalDataDir, experimentalDataFile)
    expConditionsDF = pd.read_csv(experimentalDataFilePath, sep=';',header=0)
    print('Extracted a table with ' + str(expConditionsDF.shape[0]) + ' lines and ' + str(expConditionsDF.shape[1]) + ' columns.')
    
    # Cleaning the table
    try:
        for c in expConditionsDF.columns:
            if 'Unnamed' in c:
                expConditionsDF = expConditionsDF.drop([c], axis=1)
        expConditionsDF = expConditionsDF.convert_dtypes()

        listTextColumns = []
        for col in expConditionsDF.columns:
            try:
                if expConditionsDF[col].dtype == 'string':
                    listTextColumns.append(col)
            except:
                aaaa=0
                #Ok

        expConditionsDF[listTextColumns] = expConditionsDF[listTextColumns].apply(lambda x: x.str.replace(',','.'))

        expConditionsDF['scale pixel per um'] = expConditionsDF['scale pixel per um'].astype(float)
        try:
            expConditionsDF['optical index correction'] = \
                      expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[0]).astype(float) \
                    / expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[1]).astype(float)
        except:
            print('optical index correction already in ' + str(expConditionsDF['optical index correction'].dtype) + ' type.')

        expConditionsDF['magnetic field correction'] = expConditionsDF['magnetic field correction'].astype(float)
        expConditionsDF['with fluo images'] = expConditionsDF['with fluo images'].astype(bool)

        try:
            expConditionsDF['ramp field'] = \
            expConditionsDF['ramp field'].apply(lambda x: [x.split(';')[0], x.split(';')[1]] if not pd.isnull(x) else [])
        except:
            aaaa=0
            #Ok

        dateExemple = expConditionsDF.loc[expConditionsDF.index[1],'date']

        if re.match(dateFormatExcel, dateExemple):
            print('dates corrected')
            expConditionsDF.loc[1:,'date'] = expConditionsDF.loc[1:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])        
        
    
    
    except:
        print('Unexpected bug with the cleaning step')
        
    expConditionsDF['beads bright spot delta'] = 1.2*(expConditionsDF['bead type'] != 'M450')
    
    cols = expConditionsDF.columns
    nCols = len(cols)
    listCols = np.copy(cols.values)
    listCols[14] = 'beads bright spot delta'
    for i in range(15,nCols):
        listCols[i] = cols.values[i-1]
    print(listCols)
    print(len(listCols))
        

    if save:
        saveName = 'ExperimentalConditions.csv'
        savePath = os.path.join(experimentalDataDir, saveName)
        expConditionsDF.to_csv(savePath, sep=';')

    expConditionsDF['manipID'] = expConditionsDF['date'] + '_' + expConditionsDF['manip']
    
    return(expConditionsDF)

In [18]:
modifyExperimentalConditions(save=False).head()

Extracted a table with 35 lines and 22 columns.
optical index correction already in Float64 type.
['date' 'manip' 'experimentType' 'drug' 'substrate'
 'objective magnification' 'scale pixel per um' 'objective immersion'
 'optical index correction' 'magnetic field correction' 'cell type'
 'cell subtype' 'bead type' 'bead diameter' 'beads bright spot delta'
 'normal field' 'ramp field' 'compression duration' 'with fluo images'
 'bacteria' 'comments']
21


,date,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,...,bead type,bead diameter,normal field,ramp field,compression duration,with fluo images,bacteria,comments,beads bright spot delta,manipID
0,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,100X,15.8,oil,0.875,1.15,...,M450,4503,5,['3'. '40'],1s,False,<NA>,<NA>,0.0,DEFAULT_DEFAULT
1,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,0.0,20-08-04_M1
2,20-08-04,M2,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,0.0,20-08-04_M2
3,20-08-05,M1,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,0.0,20-08-05_M1
4,20-08-05,M2,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,0.0,20-08-05_M2
